In [18]:
import numpy as np
import torch
import torch.nn as nn
import math

Требуется реализовать функцию батч-нормализации без использования стандартной функции со следующими упрощениями:

* Параметр Бета принимается равным 0.
* Параметр Гамма принимается равным 1.
* Функция должна корректно работать только на этапе обучения.
* Вход имеет размерность число элементов в батче * длина каждого инстанса.

In [ ]:
def custom_batch_norm1d(input_tensor, eps):
#     среднее значение
    mean = torch.mean(input_tensor, dim=0)
#     дисперсия
    var = torch.var(input_tensor, dim=0, unbiased=False)
    normed_tensor = (input_tensor - mean)/(torch.sqrt(var + eps))
    return normed_tensor


input_tensor = torch.Tensor([[0.0, 0, 1, 0, 2], [0, 1, 1, 0, 10]])
batch_norm = nn.BatchNorm1d(input_tensor.shape[1], affine=False)


all_correct = True
for eps_power in range(10):
    eps = np.power(10., -eps_power)
    batch_norm.eps = eps
    batch_norm_out = batch_norm(input_tensor)
    custom_batch_norm_out = custom_batch_norm1d(input_tensor, eps)

    all_correct &= torch.allclose(batch_norm_out, custom_batch_norm_out)
    all_correct &= batch_norm_out.shape == custom_batch_norm_out.shape
print(all_correct)

True


Требуется реализовать функцию батч-нормализации без использования стандартной функции со следующими упрощениями:

* Функция должна корректно работать только на этапе обучения.
* Вход имеет размерность число элементов в батче * длина каждого инстанса.

In [ ]:
# gamma = weight
# betta = bias

input_size = 7
batch_size = 5
input_tensor = torch.randn(batch_size, input_size, dtype=torch.float)

eps = 1e-3

def custom_batch_norm1d(input_tensor, weight, bias, eps):
    mean = torch.mean(input_tensor, dim=0)
    var = torch.var(input_tensor, dim=0, unbiased=False)
    normed_tensor = ((input_tensor - mean)/(torch.sqrt(var + eps))*weight)+bias
    return normed_tensor


batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm_out = batch_norm(input_tensor)
custom_batch_norm_out = custom_batch_norm1d(input_tensor, batch_norm.weight.data, batch_norm.bias.data, eps)
print(torch.allclose(batch_norm_out, custom_batch_norm_out) \
      and batch_norm_out.shape == custom_batch_norm_out.shape)

True


Реализуем работу слоя батч-нормализации на этапе предсказания.

На этом этапе вместо статистик по батчу будем использовать экспоненциально сглаженные статистики из истории обучения слоя.

В данном шаге вам требуется реализовать полноценный класс батч-нормализации без использования стандартной функции, 
принимающий на вход двумерный тензор. Осторожно, расчёт дисперсии ведётся по смещенной выборке, а расчет скользящего среднего 
по несмещенной.

In [ ]:
input_size = 3
batch_size = 5
eps = 1e-1


class CustomBatchNorm1d:
    def __init__(self, weight, bias, eps, momentum):
        self.weight = weight
        self.bias = bias
        self.eps = eps
        self.momentum = momentum
        self.mean = 0
        self.var = 1
        self.train = True

        
    def __call__(self, input_tensor):
        if self.train == True:
            mean = torch.mean(input_tensor, dim=0)
            var = torch.var(input_tensor, dim=0, unbiased=False)
            
            self.mean = (1-self.momentum) * mean + self.momentum * self.mean
            self.var = (1-self.momentum) * var * (batch_size/(batch_size-1)) + self.momentum * self.var
            
            normed_tensor = ((input_tensor - mean)/(torch.sqrt(var + self.eps)) * self.weight) + self.bias
        
        else:
            normed_tensor = ((input_tensor - self.mean)/(torch.sqrt(self.var + self.eps)) * self.weight) + self.bias
        
        return normed_tensor

    
    def eval(self):
        # В этом методе реализуем переключение в режим предикта.
        self.train = False


batch_norm = nn.BatchNorm1d(input_size, eps=eps)
batch_norm.bias.data = torch.randn(input_size, dtype=torch.float)
batch_norm.weight.data = torch.randn(input_size, dtype=torch.float)
batch_norm.momentum = 0.5

custom_batch_norm1d = CustomBatchNorm1d(batch_norm.weight.data,
                                        batch_norm.bias.data, eps, batch_norm.momentum)

all_correct = True

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output, atol=1e-04) \
        and norm_output.shape == custom_output.shape

batch_norm.eval()
custom_batch_norm1d.eval()

for i in range(8):
    torch_input = torch.randn(batch_size, input_size, dtype=torch.float)
    norm_output = batch_norm(torch_input)
    custom_output = custom_batch_norm1d(torch_input)
    all_correct &= torch.allclose(norm_output, custom_output, atol=1e-04) \
        and norm_output.shape == custom_output.shape
print(all_correct)

True


### BatchNorm

Реализуем батч-норм слой для четырехмерного входа (например, батч из многоканальных двумерных картинок) без использования 
стандартной реализации со следующими упрощениями:
* Параметр Бета = 0.
* Параметр Гамма = 1.

Функция должна корректно работать только на этапе обучения.

In [ ]:
eps = 1e-3

input_channels = 3
batch_size = 3
height = 10
width = 10

batch_norm_2d = nn.BatchNorm2d(input_channels, affine=False, eps=eps)

input_tensor = torch.randn(batch_size, input_channels, height, width, dtype=torch.float)

def custom_batch_norm2d(input_tensor, eps):
    mean = torch.cat([torch.mean(input_tensor[:,i,:,:]).unsqueeze(0) 
                      for i in range(input_channels)], dim=0)
    
    var = torch.cat([torch.var(input_tensor[:,i,:,:], unbiased=False).unsqueeze(0) 
                     for i in range(input_channels)], dim=0)
    
    normed_tensor = torch.cat([((input_tensor[:,i,:,:] - mean[i])/torch.sqrt(var[i] + eps)).unsqueeze(1) 
                               for i in range(input_channels)], dim=1)
    return normed_tensor


norm_output = batch_norm_2d(input_tensor)
custom_output = custom_batch_norm2d(input_tensor, eps)
print(torch.allclose(norm_output, custom_output) and norm_output.shape == custom_output.shape)

True


### LayerNorm

Реализуем нормализацию "по каналу" без использования стандартного слоя со следующими упрощениями:

* Параметр Бета = 0.
* Параметр Гамма = 1.
* Требуется реализация только этапа обучения.

Нормализация делается по всем размерностям входа, кроме нулевой.
Обратите внимание, что размерность входа на данном шаге не фиксирована.
Уточним, что в слое нормализации "по каналу" статистики считаются по всем размерностям, кроме нулевой.

In [ ]:
eps = 1e-10


def custom_layer_norm(input_tensor, eps):
    mean = torch.cat([torch.mean(input_tensor[i]).unsqueeze(0) 
                      for i in range(0, input_tensor.size()[0])], dim=0)
    
    var = torch.cat([torch.var(input_tensor[i], unbiased=False).unsqueeze(0) 
                     for i in range(0, input_tensor.size()[0])], dim=0)
    
    normed_tensor = torch.cat([((input_tensor[i] - mean[i])/torch.sqrt(var[i] + eps)).unsqueeze(0) 
                               for i in range(0, input_tensor.size()[0])], dim=0)
    return normed_tensor


all_correct = True
for dim_count in range(3, 9):
    input_tensor = torch.randn(*list(range(3, dim_count + 2)), dtype=torch.float)
    layer_norm = nn.LayerNorm(input_tensor.size()[1:], elementwise_affine=False, eps=eps)

    norm_output = layer_norm(input_tensor)
    custom_output = custom_layer_norm(input_tensor, eps)

    all_correct &= torch.allclose(norm_output, custom_output, 1e-2)
    all_correct &= norm_output.shape == custom_output.shape
print(all_correct)

True


Найти функцию активации, которая приводит к наименьшему затуханию градиента. 

In [25]:
seed = int(input())
np.random.seed(seed)
torch.manual_seed(seed)

NUMBER_OF_EXPERIMENTS = 200

class SimpleNet(torch.nn.Module):
    def __init__(self, activation):
        super().__init__()

        self.activation = activation
        self.fc1 = torch.nn.Linear(1, 1, bias=False)  # one neuron without bias
        self.fc1.weight.data.fill_(1.)  # init weight with 1
        self.fc2 = torch.nn.Linear(1, 1, bias=False)
        self.fc2.weight.data.fill_(1.)
        self.fc3 = torch.nn.Linear(1, 1, bias=False)
        self.fc3.weight.data.fill_(1.)

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        return x

    def get_fc1_grad_abs_value(self):
        np.random.seed(seed)
        torch.manual_seed(seed)
        return torch.abs(self.fc1.weight.grad)

def get_fc1_grad_abs_value(net, x):
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    output = net.forward(x)
    output.backward()  

  # no loss function. Pretending that we want to minimize output
  # In our case output is scalar, so we can calculate backward
    
    fc1_grad = net.get_fc1_grad_abs_value().item()
    net.zero_grad()
    return fc1_grad

  # Trying different activations to get biggest gradient
functions = ['ELU', 'Hardtanh', 'LeakyReLU', 'LogSigmoid',
               'PReLU', 'ReLU', 'ReLU6', 'RReLU', 'SELU', 'CELU',
               'Sigmoid', 'Softplus', 'Softshrink','Softsign',
               'Tanh', 'Tanhshrink', 'Hardshrink']

for current_name in functions:
  np.random.seed(seed)
  torch.manual_seed(seed)

  net = SimpleNet(activation=eval("torch.nn.{}()".format(current_name)))

fc1_grads = []
for x in torch.randn((NUMBER_OF_EXPERIMENTS, 1)):
  fc1_grads.append(get_fc1_grad_abs_value(net, x))

print(np.mean(fc1_grads))

11
0.7372176972031593


Рассчитать градиенты весов сети для случая a=tanh и для случая a=ReLU

In [19]:
def ReLU(x):
    return max(0, x)

def dReLU(x):
    if x > 0:
        return 1
    return 0

# Tanh activation
t1 =round((1 - math.tanh(math.tanh(math.tanh(math.tanh(100)))) ** 2) * (1 - math.tanh(math.tanh(math.tanh(100))) ** 2) 
          * (1 - math.tanh(math.tanh(100)) ** 2) * (1 - math.tanh(100) ** 2) * 100, 3)
t2 = round((1 - math.tanh(math.tanh(math.tanh(math.tanh(100)))) ** 2) * (1 - math.tanh(math.tanh(math.tanh(100))) ** 2) 
           * (1 - math.tanh(math.tanh(100)) ** 2) * math.tanh(100), 3)
t3 = round((1 - math.tanh(math.tanh(math.tanh(math.tanh(100)))) ** 2) * (1 - math.tanh(math.tanh(math.tanh(100))) ** 2)
      * math.tanh(math.tanh(100)), 3)
t4 = round((1 - math.tanh(math.tanh(math.tanh(math.tanh(100)))) ** 2) * math.tanh(math.tanh(math.tanh(100))), 3)
# ReLU activation 
r1 = round(dReLU(ReLU(ReLU(ReLU(100)))) * dReLU(ReLU(ReLU(100))) * dReLU(ReLU(100)) * dReLU(100) * 100, 3)
r2 = round(dReLU(ReLU(ReLU(ReLU(100)))) * dReLU(ReLU(ReLU(100))) * dReLU(ReLU(100)) * ReLU(100), 3)
r3 = round(dReLU(ReLU(ReLU(ReLU(100)))) * dReLU(ReLU(ReLU(100))) * ReLU(ReLU(100)), 3)
r4 = round(dReLU(ReLU(ReLU(ReLU(100)))) * ReLU(ReLU(ReLU(100))), 3)
answer1, answer2 = [t1, t2, t3, t4], [r1, r2, r3, r4]

print(answer1, answer2, sep=',')

[0.0, 0.168, 0.304, 0.436],[100, 100, 100, 100]
